In [ ]:
!pip install peewee pytorch-lightning
!wget https://storage.googleapis.com/chesspic/datasets/2021-07-31-lichess-evaluations-37MM.db.gz
!gzip -d "2021-07-31-lichess-evaluations-37MM.db.gz"
!pip install python-chess

In [ ]:
from peewee import *
import base64

db = SqliteDatabase('2021-07-31-lichess-evaluations-37MM.db')

class Evaluations(Model):
  id = IntegerField()
  fen = TextField()
  binary = BlobField()
  eval = FloatField()

  class Meta:
    database = db

  def binary_base64(self):
    return base64.b64encode(self.binary)
db.connect()

LABEL_COUNT = 37164639

In [17]:
import chess
import numpy as np 

GRID_DIMENSION = 8

def fen_to_binary_encoding(fen: str) -> float:
    """
    Converts a string in Forsyth-Edwards Chess Notation into binary encoding (length of 768) which is readable by the deep learning model.
    """
    # get the bitboards for each team from the board
    board = chess.Board(fen)
    black_squares, white_squares = board.occupied_co

    # Create the bitboards for each individual type of chess piece per team
    b_pawn_bitboard = black_squares & board.pawns
    b_knight_bitboard = black_squares & board.knights
    b_bishop_bitboard = black_squares & board.bishops
    b_rook_bitboard = black_squares & board.rooks
    b_queen_bitboard = black_squares & board.queens
    b_king_bitboard = black_squares & board.kings
    w_pawn_bitboard = white_squares & board.pawns
    w_knight_bitboard = white_squares & board.knights
    w_bishop_bitboard = white_squares & board.bishops
    w_rook_bitboard = white_squares & board.rooks
    w_queen_bitboard = white_squares & board.queens
    w_king_bitboard = white_squares & board.kings

    # Combine the bitboards for each chess piece into a single array
    bitboards = np.array(
        [
            b_pawn_bitboard,
            b_knight_bitboard,
            b_bishop_bitboard,
            b_rook_bitboard,
            b_queen_bitboard,
            b_king_bitboard,
            w_pawn_bitboard,
            w_knight_bitboard,
            w_bishop_bitboard,
            w_rook_bitboard,
            w_queen_bitboard,
            w_king_bitboard,
        ],
        dtype=np.uint64,
    )

    # add a new dimension to the bitboards array to make bitwise operations simpler for conversion into binary
    bitboards = np.asarray(bitboards, dtype=np.uint64)[:, np.newaxis]
    shift_amounts = GRID_DIMENSION * np.arange(
        GRID_DIMENSION - 1, -1, -1, dtype=np.uint64
    )
    # shift the bits in the bitboards array by the shift amounts and convert to uint8 data type
    binary = (bitboards >> shift_amounts).astype(np.uint8)
    # converts the binary values to a 1D array of individual bits
    binary = np.unpackbits(binary, bitorder="little")
    # returns a float data type to be used by the deep learning model
    return binary.astype(np.single)

In [25]:
import os
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, IterableDataset, random_split
import pytorch_lightning as pl
from random import randrange

class EvaluationDataset(IterableDataset):
  def __init__(self, count):
    self.count = count
  def __iter__(self):
    return self
  def __next__(self):
    idx = randrange(self.count)
    return self[idx]
  def __len__(self):
    return self.count
  def __getitem__(self, idx):
    eval = Evaluations.get(Evaluations.id == idx+1)
    bin = fen_to_binary_encoding(eval.fen)
    eval.eval = max(eval.eval, -15)
    eval.eval = min(eval.eval, 15)
    ev = np.array([eval.eval]).astype(np.single) 
    return {'binary':bin, 'eval':ev}    

dataset = EvaluationDataset(count=10000)

In [ ]:
import time
from collections import OrderedDict

class EvaluationModel(pl.LightningModule):
  def __init__(self,learning_rate=1e-3,batch_size=1024,layer_count=10):
    super().__init__()
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    layers = []
    for i in range(layer_count-1):
      layers.append((f"linear-{i}", nn.Linear(768, 768)))
      layers.append((f"relu-{i}", nn.ReLU()))
    layers.append((f"linear-{layer_count-1}", nn.Linear(768, 1)))
    self.seq = nn.Sequential(OrderedDict(layers))

  def forward(self, x):
    return self.seq(x)

  def training_step(self, batch, batch_idx):
    x, y = batch['binary'], batch['eval']
    y_hat = self(x)
    loss = F.l1_loss(y_hat, y)
    self.log("train_loss", loss)
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

  def train_dataloader(self):
    dataset = EvaluationDataset(count=1000000)
    return DataLoader(dataset, batch_size=self.batch_size, num_workers=1, pin_memory=True)

configs = [
           {"layer_count": 2, "batch_size": 1024},
           ]
for config in configs:
  version_name = f'{int(time.time())}-batch_size-{config["batch_size"]}-layer_count-{config["layer_count"]}'
  logger = pl.loggers.TensorBoardLogger("lightning_logs", name="chessml", version=version_name)
  trainer = pl.Trainer(gpus=1,precision=16,max_epochs=1,auto_lr_find=True,logger=logger)
  model = EvaluationModel(layer_count=config["layer_count"],batch_size=config["batch_size"],learning_rate=1e-3)
  trainer.fit(model)
  break

In [40]:
torch.save(model.state_dict(), "/chkpt.pt")